In [27]:
import numpy as np
import pandas as pd

from collections import Counter
from typing import List, Dict

import matplotlib.pyplot as plt
%matplotlib inline


Constants

In [28]:
RANDOM_SEED = 5
DATA_PATH = "raw_data"
AK_FILE = "AnnaKarenina"
WNP_RU_FILE = "WarAndPeace"
WNP_EN_FILE = "WarAndPeaceEng"
TXT = ".txt"

np.random.seed(RANDOM_SEED)


Data loading

In [30]:
wnp_ru = None
with open(f"{DATA_PATH}/{WNP_RU_FILE}{TXT}", "r", encoding="utf-8") as f:
    wnp_ru = f.read()
assert wnp_ru is not None


In [34]:
print(type(wnp_ru))
print(len(wnp_ru))


<class 'str'>
717873
